In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 9 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

19 December 2022 ; 16:22:26


# Features, model

In [3]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP']#,'GS'] # GS !!!
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,12,0,5.8,1.1,2.0,0.542,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,1,1
1,Bobby Portis,PF,27,MIL,29,8,26.5,6.1,12.1,0.504,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,0,1
2,Brook Lopez,C,34,MIL,28,28,30.5,5.1,10.7,0.480,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,0,0
3,George Hill,PG,36,MIL,29,0,19.8,1.6,3.8,0.427,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,0,1
4,Giannis Antetokounmpo,PF,28,MIL,24,24,32.8,11.1,21.0,0.527,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,0,0


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Nikola Jokić,DEN,32.5,9.776923,16.715385,25.2,2,0.734575
1,Giannis Antetokounmpo,MIL,27.8,6.491667,10.933333,30.6,1,0.658170
2,Luka Dončić,DAL,30.8,9.718519,15.488889,33.0,9,0.572007
3,Joel Embiid,PHI,30.6,7.790000,14.760000,33.3,5,0.516050
4,Stephen Curry,GSW,26.7,8.200000,13.561538,30.0,11,0.363260


In [7]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)